### FUNCTIONS / LIBRARIES

In [70]:
# Functions Optmization

from Optimization import prepare_vehicles_with_stats
from Optimization import spatial_optimization_pipeline
from Optimization import run_fairness_pipeline
from Optimization import run_max_coverage_pipeline
from Optimization import combine_optimized_dfs
from Optimization import compute_combined_optimization_scores
from Optimization import select_random_vehicles
from Optimization import vehicle_optimization_stats_pipeline
from Optimization import extract_string_lists # Done (call if needed)
from Optimization import plot_vehicles_by_group # Done (call if needed)
from Optimization import prepare_selected_vehicles_from_combined 
from Optimization import master_function_analysis
from Optimization import visualization_master_function
from Optimization import pipeline_plot_frequency

import warnings
import geopandas as gpd
import pandas as pd

In [71]:
# Set the warning filter to ignore SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)


### RAW DATA / PARAMETERS 

In [72]:
# Full CBS
cbs_full = gpd.read_file('data/temp/full_cbs.gpkg')

# CIty Stats

city_stats = pd.read_csv('data/city_stats_amsterdam.csv')

# Gruped by points 
points_gdf = gpd.read_file('data/temp/grouped_by_points_GVB.gpkg')

# Parameters
City = 'Amsterdam'

# City Boundary 
city_geo = gpd.read_file("data/Gemeente2.geojson")

### PROCESS

In [66]:
# Usage Create Vehicles with Stats
vehicles_stats = prepare_vehicles_with_stats(points_gdf, cbs_full)

In [67]:
vehicles_stats

,uni_id,route_id,route_type_left,crs28922_list,min_new_timest,max_new_timest,count,A_inhab,A_0_15,A_15_25,A_25_45,A_45_65,A_65+,A_nederlan,A_west_mig,A_n_west_m,G_woz_woni,geometry
88,1921_GVB,"[91905, 91947, 91949, 91910]",3.0,"[E1237N4866, E1236N4866, E1236N4867, E1237N486...",1710480607,1710564835,9625,99240.0,11563,13700,40159,21918,11900,41808,21493,35939,420.1,"MULTIPOINT (118631.03 485722.032, 118640.268 4..."
222,2268_GVB,"[91907, 91909, 91945, 91952, 91925]",3.0,"[E1168N4860, E1169N4860, E1168N4861, E1175N486...",1710480723,1710566942,6980,98335.0,12261,10825,40772,22928,11549,43462,21721,33152,482.0,"MULTIPOINT (118542.804 486393.471, 118495.082 ..."
74,1906_GVB,"[91905, 91946, 91950, 91945]",3.0,"[E1175N4863, E1176N4863, E1175N4864, E1176N486...",1710485694,1710566926,7593,94045.0,10943,12348,39690,20536,10528,39678,21050,33317,437.6,"MULTIPOINT (120096.393 483777.232, 120083.482 ..."
230,2277_GVB,"[91941, 91909, 91907, 91925]",3.0,"[E1228N4914, E1227N4914, E1214N4838, E1214N483...",1710481132,1710566924,7443,91240.0,12540,10522,36090,21260,10828,37190,16604,37446,397.3,"MULTIPOINT (121360.165 488482.56, 121351.182 4..."
75,1907_GVB,"[91949, 91910, 91951]",3.0,"[E1237N4866, E1236N4866, E1236N4867, E1237N486...",1710484195,1710564419,9173,87670.0,10094,12714,35211,19595,10056,36336,18303,33031,415.5,"MULTIPOINT (124042.301 486022.129, 124048.155 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,3019_GVB,[91912],0.0,"[E1197N4818, E1196N4818, E1197N4819, E1196N481...",1710482736,1710545162,3261,2160.0,221,213,1039,343,344,823,673,664,468.6,"MULTIPOINT (119686.976 481557.273, 119686.828 ..."
236,3008_GVB,[91912],0.0,"[E1197N4818, E1196N4818, E1197N4819, E1196N481...",1710480718,1710546069,3598,2160.0,221,213,1039,343,344,823,673,664,468.6,"MULTIPOINT (119979.633 483780.189, 119979.633 ..."
238,3022_GVB,[91912],0.0,"[E1197N4818, E1196N4818, E1197N4819, E1196N481...",1710481125,1710544260,3510,2160.0,221,213,1039,343,344,823,673,664,468.6,"MULTIPOINT (119686.978 481562.803, 119685.868 ..."
173,2108_GVB,[91901],0.0,"[E1186N4863, E1185N4863, E1186N4864, E1185N4864]",1710566837,1710566937,21,820.0,91,93,389,180,67,388,145,287,350.5,"MULTIPOINT (118638.992 486396.038, 118638.992 ..."


In [ ]:
# Export Vehicle Stats
# vehicles_stats.to_file('data/temp/vehicles_stats_AMS_50buffer_1503.gpkg', driver='GPKG')

In [5]:
# Selecte Vehicles for Spatial Optimization
optimized_spatial, filtered_vehicles, max_space_vehicles = spatial_optimization_pipeline(points_gdf, cbs_full, vehicles_stats, coverage_threshold=3, top_n=10)

In [73]:
# Selecte Vehicles for Fairness Optimization
closest_simple, closest_relative, closest_absolute, df_area_stats, df_opts, fair_vehicles = run_fairness_pipeline(vehicles_stats, city_stats, n=10)

In [74]:
# Selecte Vehicles for Max Coverage Optimization

tops, cbs_lists, max_number, gdf_filtered, max_pop_vehicles = run_max_coverage_pipeline(vehicles_stats, cbs_full, n=10)

max_A_inhab       = tops['max_A_inhab']
max_A_old         = tops['max_A_65+']
max_A_young       = tops['max_A_0_15']
max_A_dutch       = tops['max_A_nederlan']
max_A_non_western = tops['max_A_n_west_m']
max_P_old         = tops['max_P_65+']
max_P_non_western = tops['max_P_n_west_m']
max_point_count   = tops['max_count']

In [75]:
# Create Combined First Itearation 

combined_df = combine_optimized_dfs(max_space_vehicles, max_pop_vehicles, fair_vehicles)

In [76]:
combined_df

,max_spatial,max_A_inhab,max_A_65+,max_A_0_15,max_A_nederlan,max_A_n_west_m,max_P_65+,max_P_n_west_m,max_count,closest_absolute,closest_relative,closest_simple
0,1921_GVB,1921_GVB,1921_GVB,2277_GVB,2268_GVB,2277_GVB,1423_GVB,1466_GVB,1416_GVB,1462_GVB,1462_GVB,1462_GVB
1,309_GVB,2268_GVB,2268_GVB,2268_GVB,1921_GVB,1921_GVB,322_GVB,1104_GVB,2121_GVB,1917_GVB,1917_GVB,1917_GVB
2,2268_GVB,1906_GVB,1403_GVB,2257_GVB,1906_GVB,2257_GVB,1608_GVB,1118_GVB,2041_GVB,2019_GVB,2028_GVB,2089_GVB
3,2021_GVB,2277_GVB,320_GVB,1921_GVB,1910_GVB,2265_GVB,1609_GVB,1138_GVB,1141_GVB,2089_GVB,2057_GVB,2057_GVB
4,1467_GVB,1907_GVB,2277_GVB,2265_GVB,2277_GVB,1906_GVB,1604_GVB,367_GVB,315_GVB,2057_GVB,1419_GVB,2020_GVB
5,1105_GVB,2257_GVB,1906_GVB,1906_GVB,1907_GVB,2268_GVB,1406_GVB,1465_GVB,2118_GVB,2009_GVB,1161_GVB,2009_GVB
6,310_GVB,1910_GVB,1910_GVB,1907_GVB,2257_GVB,1907_GVB,333_GVB,1125_GVB,2090_GVB,2020_GVB,2089_GVB,2060_GVB
7,2125_GVB,2265_GVB,2257_GVB,309_GVB,1403_GVB,1454_GVB,1607_GVB,1154_GVB,2081_GVB,2060_GVB,315_GVB,2039_GVB
8,1408_GVB,1403_GVB,1907_GVB,2254_GVB,320_GVB,1410_GVB,3068_GVB,1128_GVB,2098_GVB,2039_GVB,2140_GVB,2030_GVB
9,2096_GVB,1454_GVB,309_GVB,2279_GVB,2265_GVB,2254_GVB,3054_GVB,1144_GVB,1912_GVB,2077_GVB,2039_GVB,2019_GVB


In [77]:
# Combined vehicles (based on chosen columns) - not included 
scores_combined_df, top_combined_final = compute_combined_optimization_scores(combined_df)

In [78]:
# Create Random Vehicles 
random_vehicles = select_random_vehicles(vehicles_stats, n=10) 

In [79]:
print(combined_df.columns.tolist())


['max_spatial', 'max_A_inhab', 'max_A_65+', 'max_A_0_15', 'max_A_nederlan', 'max_A_n_west_m', 'max_P_65+', 'max_P_n_west_m', 'max_count', 'closest_absolute', 'closest_relative', 'closest_simple']


In [80]:
fair_vehicles

,closest_absolute,closest_relative,closest_simple
0,1462_GVB,1462_GVB,1462_GVB
1,1917_GVB,1917_GVB,1917_GVB
2,2019_GVB,2028_GVB,2089_GVB
3,2089_GVB,2057_GVB,2057_GVB
4,2057_GVB,1419_GVB,2020_GVB
5,2009_GVB,1161_GVB,2009_GVB
6,2020_GVB,2089_GVB,2060_GVB
7,2060_GVB,315_GVB,2039_GVB
8,2039_GVB,2140_GVB,2030_GVB
9,2077_GVB,2039_GVB,2019_GVB


In [81]:
# Create Selected Vehicles Stats
final_df_cells_test = vehicle_optimization_stats_pipeline(
    vehicles_stats,
    cbs_full,
    city_stats,
    max_space_vehicles,
    max_pop_vehicles,
    fair_vehicles,
    top_combined_final, # as these are not included in the final analysis
    random_vehicles
)

In [82]:
final_df_cells_test

,max_spatial,max_A_inhab,max_A_65+,max_A_0_15,max_A_nederlan,max_A_n_west_m,max_P_65+,max_P_n_west_m,max_count,fairest_absolute,fairest_relative,fairest_simple,combined_opt,random,Amsterdam
A_inhab,587505.00,877585.00,861560.00,849125.00,874315.00,853180.00,172690.00,162365.00,282840.00,331010.00,331870.00,330255.00,528490.00,239455.00,-
A_0_15,71715.00,108240.00,106600.00,110172.00,107637.00,107160.00,25197.00,23952.00,31532.00,35068.00,39775.00,34984.00,63895.00,28670.00,-
A_65+,72863.00,104777.00,106683.00,98762.00,106656.00,99245.00,30652.00,20429.00,36727.00,39983.00,40844.00,39888.00,66342.00,29685.00,-
A_nederlan,261317.00,369458.00,372323.00,355076.00,375432.00,348964.00,93171.00,36418.00,129057.00,148290.00,141403.00,147913.00,231411.00,111423.00,-
A_n_west_m,199562.00,324815.00,308370.00,324427.00,315273.00,330549.00,49174.00,106755.00,91139.00,107371.00,119512.00,107186.00,182353.00,74884.00,-
count,95856.00,82135.00,80766.00,82562.00,79197.00,81200.00,98505.00,52062.00,137532.00,102564.00,91836.00,104212.00,104689.00,61619.00,2127870
A_inhab_uniq,354185.00,256155.00,272150.00,255155.00,257180.00,230950.00,44055.00,28735.00,167980.00,116185.00,198835.00,116185.00,288955.00,153465.00,870375.0
A_0_15_uniq,45384.00,33177.00,35750.00,34090.00,33023.00,29573.00,5855.00,4122.00,19886.00,13378.00,24960.00,13378.00,37180.00,19388.00,122388
A_65+_uniq,44718.00,32140.00,34365.00,31496.00,32460.00,28481.00,7898.00,3775.00,21616.00,14898.00,24455.00,14898.00,36355.00,19508.00,112084
A_nederlan_uniq,158150.00,107268.00,116027.00,107731.00,109405.00,95536.00,22879.00,7066.00,75604.00,49270.00,82951.00,49270.00,125221.00,66656.00,382106


In [69]:
# Createa CSV with the selected vehicles
# combined_df.to_csv('data/temp/combined_df_10_AMS_1503.csv', index=False)

In [ ]:
# Usage Visualization Quick (Only if needed!)

# lists_dict = extract_string_lists(combined_df)
# lists_dict.keys()
# plot_vehicles_by_group(vehicles_stats, lists_dict, city_geo) 

In [17]:
# TO DO: FINISH PIPELINE - CREATA OPTMIZED VEHICLES, VIZ ANALYSIS MAIN (separate script?), + FREQUENCY (separate script?)

In [31]:
# THIS CAN BE A NEW SEPARAE SCRIPT VISUALISATION! 

In [35]:
# Test for Max Spatial

In [ ]:
# # Create Optmized Vehicles GDFs 
# max_space_gdf = prepare_selected_vehicles_from_combined(vehicles_stats, combined_df, column='max_spatial')
# # Create Main Analysis 
# sums_df, cbs_gdf, joined_gdf, average_stats = master_function_analysis(
#     max_space_gdf, cbs_full,50)
# # Create Main Visualization 
# fig1, fig2, fig3, fig4 = visualization_master_function(
#     max_space_gdf, cbs_full, joined_gdf, city_geo, 50, sums_df, average_stats)